In [18]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [43]:
bechdel_movies = pd.read_csv('movie_metadata_all_features.csv')

rf_features = bechdel_movies[["actor_mention_score",
                             "negative",
                             "positive",
                             "neutral",
                             "female_ratio",
                             "gender_density",
                             "bechdel_binary"]]

rf_features = rf_features.dropna()
print(rf_features)

rf_features = rf_features.to_numpy()
print(rf_features.shape)
rf_labels = rf_features[:,-1]
rf_features = rf_features[:,:-1]
print(rf_features.shape, rf_labels.shape)


      actor_mention_score  negative  positive   neutral  female_ratio  \
0                  0.2500  0.048826  0.120614  0.830560      0.150000   
1                  0.5345  0.129004  0.179076  0.691920      0.461538   
2                  0.2830  0.252773  0.112999  0.634228      0.428571   
5                  1.0000  0.331542  0.086043  0.582415      0.250000   
7                  0.2500  0.640489  0.023423  0.336088      0.285714   
...                   ...       ...       ...       ...           ...   
5188               0.2222  0.333736  0.058590  0.607674      0.294118   
5189               0.7500  0.384289  0.049881  0.565830      0.235294   
5190               0.1667  0.131713  0.176800  0.691487      0.454545   
5191               0.5200  0.317630  0.097013  0.585357      0.285714   
5193               0.0000  0.511130  0.049057  0.439813      0.375000   

      gender_density  bechdel_binary  
0           0.000000               1  
1           0.333333               1  
2     

In [44]:
x_train, x_test, y_train, y_test = train_test_split(rf_features, rf_labels,test_size=0.3)

print("After train-test splitting:")
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

After train-test splitting:
(2699, 6) (2699,) (1157, 6) (1157,)


In [45]:
rf = RandomForestClassifier(n_estimators = 100)
#rf_best = SelectFromModel(rf)
rf.fit(x_train, y_train)
#rf.get_support()

RandomForestClassifier()

In [46]:
y_pred = rf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.7199654278305964


In [22]:
"actor_mention_score",
                             "negative",
                             "positive",
                             "neutral",
                             "female_ratio",
                             "gender_density",

[3. 3. 1. ... 0. 3. 3.]


In [33]:
#"actor_mention_score"
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(np.expand_dims(x_train[:,0], axis = -1), y_train)
y_pred = rf.predict(np.expand_dims(x_test[:,0], axis = -1))
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5185825410544511


In [34]:
#negative
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(np.expand_dims(x_train[:,1], axis = -1), y_train)
y_pred = rf.predict(np.expand_dims(x_test[:,1], axis = -1))
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.39671564390665515


In [35]:
#positive
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(np.expand_dims(x_train[:,2], axis = -1), y_train)
y_pred = rf.predict(np.expand_dims(x_test[:,2], axis = -1))
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.39844425237683667


In [36]:
#neutral
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(np.expand_dims(x_train[:,3], axis = -1), y_train)
y_pred = rf.predict(np.expand_dims(x_test[:,3], axis = -1))
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.3993085566119274


In [37]:
#female cast ratio
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(np.expand_dims(x_train[:,4], axis = -1), y_train)
y_pred = rf.predict(np.expand_dims(x_test[:,4], axis = -1))
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5799481417458946


In [42]:
#gender mention density
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(np.expand_dims(x_train[:,5], axis = -1), y_train)
y_pred = rf.predict(np.expand_dims(x_test[:,5], axis = -1))
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5393258426966292
